In [1]:
import pandas as pd
d=pd.read_html('https://en.wikipedia.org/wiki/2019_in_spaceflight#Orbital_launches')

In [2]:
type(d)

list

In [3]:
df=d[3]
df.head()

Date and time (UTC)             Rocket      Flight number  \
     Date and time (UTC) Unnamed: 1_level_1 Unnamed: 2_level_1   
     Date and time (UTC) Unnamed: 1_level_2 Unnamed: 2_level_2   
0  Payload (⚀ = CubeSat)           Operator              Orbit   
1                Remarks            Remarks            Remarks   
2          January[edit]      January[edit]      January[edit]   
3    10 January17:05[20]    Long March 3B/E         3B-Y56[21]   
4    10 January17:05[20]        Chinasat 2D               CNSA   

         Launch site                                   LSP                     
  Unnamed: 3_level_1 Unnamed: 4_level_1 Unnamed: 5_level_1 Unnamed: 6_level_1  
  Unnamed: 3_level_2 Unnamed: 4_level_2 Unnamed: 5_level_2 Unnamed: 6_level_2  
0           Function        Decay (UTC)            Outcome                NaN  
1            Remarks            Remarks            Remarks                NaN  
2      January[edit]      January[edit]      January[edit]      January[edit]  
3       Xichang LC-2       Xichang LC-2               CASC               CASC  
4     Geosynchronous     Communications           In orbit        Operational

In [4]:
date=pd.DataFrame({'date':df.iloc[:,0]})
outcome=pd.DataFrame({'outcome':df.iloc[:,-1]})
df = pd.concat([date, outcome], axis=1)
df.head()

,date,outcome
0,Payload (⚀ = CubeSat),NaN
1,Remarks,NaN
2,January[edit],January[edit]
3,10 January17:05[20],CASC
4,10 January17:05[20],Operational


In [5]:
df.dropna(subset=['outcome'], how='all', inplace=True)
df.head()

,date,outcome
2,January[edit],January[edit]
3,10 January17:05[20],CASC
4,10 January17:05[20],Operational
5,11 January15:31[22],SpaceX
6,11 January15:31[22],Operational


In [6]:
check=df['outcome'].str.contains('Operational|Successful|En Route')
df=df[check]
df.head()

,date,outcome
4,10 January17:05[20],Operational
6,11 January15:31[22],Operational
11,18 January00:50:20[26],Operational
12,18 January00:50:20[26],Operational
13,18 January00:50:20[26],Operational


In [7]:
df=df.groupby((df['date'] != df['date'].shift()).cumsum().values).first()
#remove duplicates first only because the [] is link to the type of rocket so same day, same time and and same rocket has no duplicates.
df['date'] = df['date'].str.replace(r"\[.*\]","")
df['freq'] = df.groupby('date')['date'].transform('count')
# this for the frequency of same time in case mutliple launches at same time
df.head()

,date,outcome,freq
1,10 January17:05,Operational,1
2,11 January15:31,Operational,1
3,18 January00:50:20,Operational,1
4,19 January19:05,Operational,1
5,21 January05:42,Operational,1


In [8]:
def Get_Time(date):
    head = date.rstrip('0123456789:')
    tail = date[len(head):]
    if len(tail)==5:
        return "00:"+tail
    return tail
    
def Clean_date(date):
    head = date.rstrip('0123456789:')
    return head

df['time']= df['date'].apply(Get_Time)
df['date'] = df['date'].apply(Clean_date)
df.head()

,date,outcome,freq,time
1,10 January,Operational,1,00:17:05
2,11 January,Operational,1,00:15:31
3,18 January,Operational,1,00:50:20
4,19 January,Operational,1,00:19:05
5,21 January,Operational,1,00:05:42


In [9]:
import re

def Clean_date(date):
    hmap={'January':'1','February':'2','March':'3','April':'4','May':'5','June':'6','July':'7','August':'8','September':'9','October':'10','November':'11','December':'12'}
    l=date.split(" ")
    if len(l[0])==1:
        l[0]='0'+l[0]
    return '2019-'+hmap[l[1]]+'-'+l[0]
df['date'] = df['date'].apply(Clean_date) 
df.head()

,date,outcome,freq,time
1,2019-1-10,Operational,1,00:17:05
2,2019-1-11,Operational,1,00:15:31
3,2019-1-18,Operational,1,00:50:20
4,2019-1-19,Operational,1,00:19:05
5,2019-1-21,Operational,1,00:05:42


In [10]:
df.dtypes

date       object
outcome    object
freq        int64
time       object
dtype: object

In [11]:
df['date']=pd.to_datetime(df['date'])

In [12]:
columns=['date']
df2 = pd.DataFrame(columns=columns)
df2['date']=pd.Series(pd.date_range('2019-01-01', freq='D', periods=365))
df2.head()

,date
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-05


In [13]:
result = df2.merge(df,on='date',how='outer')

In [14]:
result.head()

,date,outcome,freq,time
0,2019-01-01,NaN,NaN,NaN
1,2019-01-02,NaN,NaN,NaN
2,2019-01-03,NaN,NaN,NaN
3,2019-01-04,NaN,NaN,NaN
4,2019-01-05,NaN,NaN,NaN


In [15]:
result.drop('outcome',inplace=True,axis=1)
result.head()

,date,freq,time
0,2019-01-01,NaN,NaN
1,2019-01-02,NaN,NaN
2,2019-01-03,NaN,NaN
3,2019-01-04,NaN,NaN
4,2019-01-05,NaN,NaN


In [16]:
import numpy as np
result.freq.fillna(0, inplace=True)

In [17]:
result.head()

,date,freq,time
0,2019-01-01,0.0,NaN
1,2019-01-02,0.0,NaN
2,2019-01-03,0.0,NaN
3,2019-01-04,0.0,NaN
4,2019-01-05,0.0,NaN


In [18]:
result.dtypes

date    datetime64[ns]
freq           float64
time            object
dtype: object

In [19]:
result.time.fillna("00:00:00", inplace=True)

In [20]:
result.head()
result['date'] = result.date.astype(str)

In [21]:
result.dtypes

date     object
freq    float64
time     object
dtype: object

In [22]:
result['date_m'] = result['date'].map(str)+" "+result['time'].map(str)
result

,date,freq,time,date_m
0,2019-01-01,0.0,00:00:00,2019-01-01 00:00:00
1,2019-01-02,0.0,00:00:00,2019-01-02 00:00:00
2,2019-01-03,0.0,00:00:00,2019-01-03 00:00:00
3,2019-01-04,0.0,00:00:00,2019-01-04 00:00:00
4,2019-01-05,0.0,00:00:00,2019-01-05 00:00:00
5,2019-01-06,0.0,00:00:00,2019-01-06 00:00:00
6,2019-01-07,0.0,00:00:00,2019-01-07 00:00:00
7,2019-01-08,0.0,00:00:00,2019-01-08 00:00:00
8,2019-01-09,0.0,00:00:00,2019-01-09 00:00:00
9,2019-01-10,1.0,00:17:05,2019-01-10 00:17:05


In [23]:
import datetime
test_date = datetime.datetime.strptime("2019-01-01 00:00:00", '%Y-%m-%d %H:%M:%S')
iso_date = test_date.isoformat()

In [24]:
iso_date

'2019-01-01T00:00:00'

In [25]:
import datetime

def ISO(date):
    d=datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return d.isoformat()
result['date_m'] = result['date_m'].apply(ISO) 
result.head()

,date,freq,time,date_m
0,2019-01-01,0.0,00:00:00,2019-01-01T00:00:00
1,2019-01-02,0.0,00:00:00,2019-01-02T00:00:00
2,2019-01-03,0.0,00:00:00,2019-01-03T00:00:00
3,2019-01-04,0.0,00:00:00,2019-01-04T00:00:00
4,2019-01-05,0.0,00:00:00,2019-01-05T00:00:00


In [26]:
d_f=pd.concat([result['date_m'],result['freq']],keys=['date','value'],axis=1)
d_f

,date,value
0,2019-01-01T00:00:00,0.0
1,2019-01-02T00:00:00,0.0
2,2019-01-03T00:00:00,0.0
3,2019-01-04T00:00:00,0.0
4,2019-01-05T00:00:00,0.0
5,2019-01-06T00:00:00,0.0
6,2019-01-07T00:00:00,0.0
7,2019-01-08T00:00:00,0.0
8,2019-01-09T00:00:00,0.0
9,2019-01-10T00:17:05,1.0


In [27]:
d_f.to_csv('output.csv', encoding='utf-8', index=False)